<a href="https://colab.research.google.com/github/JVachet/DUDS/blob/master/GDRIVE_2020_exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework pandas

<table align="left">
    <tr>
    <td><a href="https://colab.research.google.com/github/airnandez/numpandas/blob/master/exam/2020-exam.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a></td>
    <td><a href="https://mybinder.org/v2/gh/airnandez/numpandas/master?filepath=exam%2F2020-exam.ipynb">
  <img src="https://mybinder.org/badge_logo.svg" alt="Launch Binder"/>
</a></td>
  </tr>
</table>

*Author: Fabio Hernandez*

*Last updated: 2020-03-19*

*Location:* https://github.com/airnandez/numpandas/exam

--------------------
## Instructions

For this excercise we will use a public dataset titled **"Demandes de valeurs foncières géolocalisées"** available [here](https://www.data.gouv.fr/fr/datasets/demandes-de-valeurs-foncieres-geolocalisees/). This dataset contains information about registered real state transactions (_mutations immobilières_) in France over several years. There is a file per year. The structure of the files and the semantics of each column are documented at its source.

For your convenience, this notebook is prepared with code for downloading the dataset from its source, loading it into memory as a **pandas** dataframe and with some cleaning and helper functions. Your mission is execute the provided cells and to write the code to answer the questions below.

You must not modify the code provided. You must provide code for answering the questions, following the instructions for each one of them.

When you have finished, please save your notebook in the form of a `.ipynb` file and send it by e-mail to your instructor according to the indications you received by e-mail.

---------------------
## Dependencies

In [0]:
import datetime
import os
import glob

In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.__version__

'1.0.3'

In [4]:
import numpy as np
np.__version__

'1.18.2'

------
## Download the dataset

Define a helper function for downloading data to a local file:

In [0]:
import requests
import shutil

def download_file(url, path):
    with requests.get(url, stream=True) as r:
        with open(path, 'wb') as f:
            shutil.copyfileobj(r.raw, f)
    return

Download the data files, one per year, for the period 2014-2019, both inclusive. We store the downloaded data in the directory `../data` relative to the location of this notebook. If a file has been already been downloaded, don't download it again. The total amount of data to download is about 400 MB.

In [6]:
# Create destination directory
os.makedirs(os.path.join('..', 'data'), exist_ok=True)

# Download files
data_source = "https://cadastre.data.gouv.fr/data/etalab-dvf/latest/csv"

for year in range(2014, 2020):
    # Build the URL and the destination file path
    url = f'{data_source}/{year}/full.csv.gz'
    path = os.path.join('..', 'data', f'{year}-mutations-immobilieres.csv.gz')
    
    # If file already exists don't download it again
    if not os.path.isfile(path) :
        print(f'downloading {url} to {path}')
        download_file(url, path)

downloading https://cadastre.data.gouv.fr/data/etalab-dvf/latest/csv/2014/full.csv.gz to ../data/2014-mutations-immobilieres.csv.gz
downloading https://cadastre.data.gouv.fr/data/etalab-dvf/latest/csv/2015/full.csv.gz to ../data/2015-mutations-immobilieres.csv.gz
downloading https://cadastre.data.gouv.fr/data/etalab-dvf/latest/csv/2016/full.csv.gz to ../data/2016-mutations-immobilieres.csv.gz
downloading https://cadastre.data.gouv.fr/data/etalab-dvf/latest/csv/2017/full.csv.gz to ../data/2017-mutations-immobilieres.csv.gz
downloading https://cadastre.data.gouv.fr/data/etalab-dvf/latest/csv/2018/full.csv.gz to ../data/2018-mutations-immobilieres.csv.gz
downloading https://cadastre.data.gouv.fr/data/etalab-dvf/latest/csv/2019/full.csv.gz to ../data/2019-mutations-immobilieres.csv.gz


Check what files we have for our analysis:

In [7]:
file_paths = glob.glob(os.path.join('..', 'data', '*-mutations-immobilieres.csv.gz'))
print('\n'.join(f for f in file_paths))

../data/2015-mutations-immobilieres.csv.gz
../data/2016-mutations-immobilieres.csv.gz
../data/2018-mutations-immobilieres.csv.gz
../data/2014-mutations-immobilieres.csv.gz
../data/2019-mutations-immobilieres.csv.gz
../data/2017-mutations-immobilieres.csv.gz


---------------------
## Load the dataset

Load the dataset (i.e. all the files `../data/*-mutations-immobilieres.csv.gz`) to a **pandas** dataframe. Here we select the columns we want to load. The information about the format and contents of each column is available [here](https://www.data.gouv.fr/fr/datasets/demandes-de-valeurs-foncieres-geolocalisees/). Please make sure you are familiar with that information which you will need for analysing the data:

In [0]:
# These are the names of the columns present in the source files.
# We are not interested in analysing the commented columns, so we don't tell
# pandas to not load them
columns = (
    'id_mutation',
    'date_mutation',
    'numero_disposition',
    'nature_mutation',
    'valeur_fonciere',
    'adresse_numero',
    'adresse_suffixe',
    'adresse_nom_voie',
    'adresse_code_voie',
    'code_postal',
    'code_commune',
    'nom_commune',
    'code_departement',
#   'ancien_code_commune',
#   'ancien_nom_commune',
#   'id_parcelle',
#   'ancien_id_parcelle',
#   'numero_volume',
    'lot1_numero',
    'lot1_surface_carrez',
    'lot2_numero',
    'lot2_surface_carrez',
    'lot3_numero',
    'lot3_surface_carrez',
    'lot4_numero',
    'lot4_surface_carrez',
    'lot5_numero',
    'lot5_surface_carrez',
    'nombre_lots',
    'code_type_local',
    'type_local',
    'surface_reelle_bati',
    'nombre_pieces_principales',
#   'code_nature_culture',
    'nature_culture',
#   'code_nature_culture_speciale',
#   'nature_culture_speciale',
    'surface_terrain',
#   'longitude',
#   'latitude'
)

# These are the types we want pandas to use for each column
column_types = {
    'id_mutation': object,
    'adresse_suffixe': str,
    'adresse_numero': str,
    'adresse_suffixe': str,
    'adresse_nom_voie': str,
    'adresse_code_voie': str,
    'code_postal': str,
    'code_commune': str,
    'code_departement': str,
    'ancien_code_commune': str,
    'ancien_nom_commune': str,
    'id_parcelle': str,
    'ancien_id_parcelle': str,
    'lot1_numero': str,
    'lot2_numero': str,
    'lot3_numero': str,
    'lot4_numero': str,
    'lot5_numero': str,
    'code_type_local': str,
    'type_local': str,
}

In [9]:
# Explicitly delete our existing dataframe, if any
try:
    del df
except NameError:
    pass

file_paths = glob.glob(os.path.join('..', 'data', '*-mutations-immobilieres.csv.gz'))
df = pd.DataFrame()
for path in sorted(file_paths):
    print(f'Loading {path}')
    df = df.append(pd.read_csv(path, usecols=columns, dtype=column_types, parse_dates=['date_mutation']))

Loading ../data/2014-mutations-immobilieres.csv.gz
Loading ../data/2015-mutations-immobilieres.csv.gz
Loading ../data/2016-mutations-immobilieres.csv.gz
Loading ../data/2017-mutations-immobilieres.csv.gz
Loading ../data/2018-mutations-immobilieres.csv.gz
Loading ../data/2019-mutations-immobilieres.csv.gz


In [10]:
# Inspect the dimensions of the dataframe
rows, columns = df.shape
print(f'This dataframe has {rows:,} rows and {columns:,} columns')

This dataframe has 15,657,831 rows and 30 columns


In [11]:
df.head(10)

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,code_commune,nom_commune,code_departement,lot1_numero,lot1_surface_carrez,lot2_numero,lot2_surface_carrez,lot3_numero,lot3_surface_carrez,lot4_numero,lot4_surface_carrez,lot5_numero,lot5_surface_carrez,nombre_lots,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,nature_culture,surface_terrain
0,2014-1,2014-01-09,1,Vente,251500.0,532,NaN,RTE DE MEZERIAT,0006,01310,01301,Polliat,01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,Maison,147.0,5.0,sols,1501.0
1,2014-2,2014-01-09,1,Vente,174500.0,22,NaN,BD MAL LECLERC,2540,01000,01053,Bourg-en-Bresse,01,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3,Dépendance,NaN,0.0,NaN,NaN
2,2014-2,2014-01-09,1,Vente,174500.0,22,NaN,BD MAL LECLERC,2540,01000,01053,Bourg-en-Bresse,01,49,94.22,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,Appartement,80.0,2.0,NaN,NaN
3,2014-3,2014-01-02,1,Vente,157500.0,1389,NaN,RTE DE STRASBOURG,0201,01440,01451,Viriat,01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,sols,24.0
4,2014-3,2014-01-02,1,Vente,157500.0,1369,NaN,RTE DE STRASBOURG,0201,01440,01451,Viriat,01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,Maison,103.0,4.0,sols,1569.0
5,2014-4,2014-01-06,2,Vente,280.5,NaN,NaN,AUX RAVETTES,B205,01250,01447,Villereversure,01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,prés,535.0
6,2014-5,2014-01-07,1,Vente,90000.0,4,NaN,RUE GEN DELESTRAINT,1650,01000,01053,Bourg-en-Bresse,01,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3,Dépendance,NaN,0.0,NaN,NaN
7,2014-5,2014-01-07,1,Vente,90000.0,4,NaN,RUE GEN DELESTRAINT,1650,01000,01053,Bourg-en-Bresse,01,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,Appartement,61.0,2.0,NaN,NaN
8,2014-6,2014-01-11,1,Vente,370000.0,215,NaN,RTE DE LA GRANDE VAVRETTE,0260,01250,01422,Tossiat,01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,Maison,220.0,8.0,sols,880.0
9,2014-6,2014-01-11,1,Vente,370000.0,NaN,NaN,A LA BALETTE,B002,01250,01422,Tossiat,01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,prés,437.0


### WARNING:

Please note that there may be several rows for the same transaction. All the rows part of a single transaction have the same identifier (i.e. the same value) in the `id_mutation` column as well as the same value in the column `valeur_fonciere`. For instance, there are two rows with the value `2014-2` in the `id_mutation` column:

In [12]:
df[df['id_mutation'] == '2014-2']

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,code_commune,nom_commune,code_departement,lot1_numero,lot1_surface_carrez,lot2_numero,lot2_surface_carrez,lot3_numero,lot3_surface_carrez,lot4_numero,lot4_surface_carrez,lot5_numero,lot5_surface_carrez,nombre_lots,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,nature_culture,surface_terrain
1,2014-2,2014-01-09,1,Vente,174500.0,22,NaN,BD MAL LECLERC,2540,01000,01053,Bourg-en-Bresse,01,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3,Dépendance,NaN,0.0,NaN,NaN
2,2014-2,2014-01-09,1,Vente,174500.0,22,NaN,BD MAL LECLERC,2540,01000,01053,Bourg-en-Bresse,01,49,94.22,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,Appartement,80.0,2.0,NaN,NaN


## Inspect the dataset

Let's see what **kind of transactions** are encoded in these records:

In [13]:
print('\n'.join(df['nature_mutation'].unique()))

Vente
Echange
Vente en l'état futur d'achèvement
Vente terrain à bâtir
Adjudication
Expropriation


And what **kind of properties** are these transactions about:

In [14]:
for t in df['type_local'].unique():
    print(t)

Maison
Dépendance
Appartement
nan
Local industriel. commercial ou assimilé


### Values for filters
Here we define some convenient constants that we can use for building masks:

In [0]:
APPARTMENT = 'Appartement'
HOUSE      = 'Maison'
BUSINESS   = 'Local industriel. commercial ou assimilé'
DEPENDANCE = 'Dépendance'

-------------------
# Questions (10 points + bonus)

---------------------
## Question N° 1

### Question 1a (1 point)
How many transactions of type sale (i.e. those with value `Vente` in the column `nature_mutation`) were registered in the period covered in the dataset?

In [0]:
# Your code here

#On extrait une vue sur les ventes seules car cette vue servira dans d'autrs questions
sales = df[df['nature_mutation']=='Vente']

sales_count = sales['id_mutation'].count()

In [17]:
print(f'There are {sales_count:,} sales in the dataset')

There are 14,138,233 sales in the dataset


### Question 1b (1 point)
How many sales were registered for each kind of property (i.e. `Maison`, `Dépendance`, `Appartement` and `Local industriel`) in the whole period?

In [0]:
# Your code here

# Plutot que de partir du DataFrame complet, j'utilise ma vue sur les ventes

house_count    = sales[sales['type_local'] == HOUSE]['id_mutation'].count()
appt_count     = sales[sales['type_local'] == APPARTMENT]['id_mutation'].count()
business_count = sales[sales['type_local'] == BUSINESS]['id_mutation'].count()
dependance_count = sales[sales['type_local'] == DEPENDANCE]['id_mutation'].count()

In [19]:
# Determine the period covered in the dataset
start_date, end_date = df['date_mutation'].min(), df['date_mutation'].max()

# Compute the percentage of sales per kind of object
house_pct    = 100.0 * (house_count/sales_count)
appt_pct     = 100.0 * (appt_count/sales_count)
business_pct = 100.0 * (business_count/sales_count)
dependance_pct = 100.0 * (dependance_count/sales_count)

# Print the report
print(f'Period covered: from {start_date:%Y-%m-%d} to {end_date:%Y-%m-%d}:')
print(f'          total:  {sales_count:>10,} sales')
print(f'         houses:  {house_count:>10,} ({house_pct:>2.0f}%)')
print(f'    appartments:  {appt_count:>10,} ({appt_pct:>2.0f}%)')
print(f'       business:  {business_count:>10,} ({business_pct:>2.0f}%)')
print(f'     dependance:  {dependance_count:>10,} ({dependance_pct:>2.0f}%)')

Period covered: from 2014-01-01 to 2019-06-30:
          total:  14,138,233 sales
         houses:   3,009,883 (21%)
    appartments:   2,493,442 (18%)
       business:     526,480 ( 4%)
     dependance:   1,834,327 (13%)


### Question 1c (2 points)
What is the total amount of money (in million €) involved in those sales? Please remember that there may be several rows for a single transaction and within a single transaction each row has the same value in the column `valeur_fonciere`. You may want to consider grouping all the rows for the same transaction.

In [0]:
# Your code here
# En premier on reduit le dataset aux données nécessaires
sales_lite = sales.filter(items=['id_mutation', 'valeur_fonciere'])
# sales_lite.head(3)

Pour regrouper les données par transaction pour que les valeurs ne soient pas doublonnées
2 solutions Possibles :
1. Faire un groupby, en forçant une fonction d'aggregation , par exemple count(), car sans fonction d'aggregation le group by ne supprime pas les doublons
2. utiliser la fonciton drop_duplicates

Ici j'utilise le drop_duplicates ,car j'utiliserais l'autre méthode pour sommer les surfaces dans les questions suivantes

In [21]:
#distinct_sales = sales_lite.groupby(['id_mutation', 'valeur_fonciere'], sort=False)[['id_mutation']].count()

distinct_sales = sales_lite.drop_duplicates()
distinct_sales.head(10)

,id_mutation,valeur_fonciere
0,2014-1,251500.0
1,2014-2,174500.0
3,2014-3,157500.0
5,2014-4,280.5
6,2014-5,90000.0
8,2014-6,370000.0
10,2014-7,95000.0
12,2014-8,80000.0
13,2014-9,205000.0
14,2014-10,156750.0


In [22]:
sales_in_million_euros = distinct_sales['valeur_fonciere'].sum()/1000000

print(f'The total amount of money in sales was {sales_in_million_euros:,.0f} million €')

The total amount of money in sales was 1,202,598 million €


-----------
## Question N° 2

### Question 2a (3 points)
Your client, a big international corporation, is looking to purchase a property for installing a retail store in the Av. des Champs Elysées, in Paris. They hire you to provide an estimation of the necessary budget to purchase a property based on the data recorded in this dataset. You should only consider transactions involving business properties with a surface bigger than 300 m², 

In [23]:
# First, build a view with the relevant data

# En premier on reduit le dataset aux données nécessaires
# Pour l'instant je n'applique que le filtre "BUSINESS", on verra pourquoi

sales_2 = sales[(sales['type_local'] == BUSINESS)]\
        [['id_mutation','adresse_numero','adresse_nom_voie','nom_commune','code_postal','code_commune','surface_reelle_bati','valeur_fonciere']]

sales_2.head(10)

,id_mutation,adresse_numero,adresse_nom_voie,nom_commune,code_postal,code_commune,surface_reelle_bati,valeur_fonciere
139,2014-63,900,RTE DE NEUVILLE,Villereversure,01250,01447,460.0,1.0
197,2014-89,536,CHE DU PETIT PLAN,Saint-Just,01250,01369,560.0,120000.0
198,2014-89,354,CHE DU PETIT PLAN,Saint-Just,01250,01369,595.0,120000.0
215,2014-96,506,GR GRANDE RUE,Attignat,01340,01024,93.0,175000.0
219,2014-96,506,GR GRANDE RUE,Attignat,01340,01024,31.0,175000.0
231,2014-99,621,RTE DE CHALON,Saint-Trivier-de-Courtes,01560,01388,646.0,70000.0
237,2014-101,5425,LES MOISSONNIERS,Saint-Nizier-le-Bouchoux,01560,01380,250.0,90000.0
286,2014-123,5321,TAILLIS DES COUARDES,Bény,01370,01038,12.0,750.0
302,2014-130,6,RUE EDGAR QUINET,Bourg-en-Bresse,01000,01053,72.0,610000.0
330,2014-140,6590,RUE DES VAREYS,Viriat,01440,01451,1307.0,1.0


**POURQUOI seulement BUSINESS**

Pourquoi n'a-t-on pas appliqué tout de suite le filtre sur les adresses ?

**Il existe des cas tordus où pour une même transaction avec une même valeur j'ai plusieurs surface et surtout plusieurs adresses comme l'exemple ci-dessous :**

| id_mutation | adresse_numero | adresse_nom_voie | nom_commune | code_postal | code_commune | surface_reelle_bati | valeur_fonciere |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 2018-17 | 5234 | AU GROS LOUP | Manziat | 01570 | 01231 | 80000.0 | 295.0 |
| 2018-17 | 5536 | AU GROS LOUP | Manziat | 01570 | 01231 | 80000.0 | 390.0 |

- Je vais donc faire 2 vues séparées :
  1. une qui calcule les surfaces_baties par transaction et valeur_fonciere pour le calcul au m²
  2. une qui regroupera les adresses par transaction


- Ensuite je joindrais sur l'identifiant de transaction
- C'est avec ces vues que je vais appliquer les filtres supplémentaires

In [24]:
# Je profite de la création de la vue adresse pour appliquer les filtres liés à l'adresse
# AV DES CHAMPS ELYSEES
# A PARIS, il y a plusieurs arrondissement donc le nom de la ville n'est pas direct
address_by_trans = sales_2[ (sales_2['adresse_nom_voie'].str.contains('AV DES CHAMPS ELYSEES', regex=False))
            & (sales_2['nom_commune'].str.contains('Paris'))]\
            [['id_mutation','adresse_numero','adresse_nom_voie','nom_commune','code_postal','code_commune']]\
            .drop_duplicates()

address_by_trans.head(10)              

,id_mutation,adresse_numero,adresse_nom_voie,nom_commune,code_postal,code_commune
2461435,2014-1054448,66,AV DES CHAMPS ELYSEES,Paris 8e Arrondissement,75008,75108
2462031,2014-1054848,66,AV DES CHAMPS ELYSEES,Paris 8e Arrondissement,75008,75108
2462145,2014-1054922,66,AV DES CHAMPS ELYSEES,Paris 8e Arrondissement,75008,75108
2462436,2014-1055043,34,AV DES CHAMPS ELYSEES,Paris 8e Arrondissement,75008,75108
2464657,2014-1056322,63,AV DES CHAMPS ELYSEES,Paris 8e Arrondissement,75008,75108
2464912,2014-1056494,76,AV DES CHAMPS ELYSEES,Paris 8e Arrondissement,75008,75108
2689541,2015-1144866,76,AV DES CHAMPS ELYSEES,Paris 8e Arrondissement,75008,75108
2690340,2015-1145372,76,AV DES CHAMPS ELYSEES,Paris 8e Arrondissement,75008,75108
2690417,2015-1145429,34,AV DES CHAMPS ELYSEES,Paris 8e Arrondissement,75008,75108
2691163,2015-1145894,63,AV DES CHAMPS ELYSEES,Paris 8e Arrondissement,75008,75108


In [25]:
# J'applique la fonction d'aggregation pour sommer les surfaces d'une même transaction
# on en profite pour renommer la colonne sommée pour ne pas confondre
#surface_by_trans = sales_2.groupby(['id_mutation','valeur_fonciere'], sort=False)[['surface_reelle_bati']].sum()

surface_by_trans = sales_2.groupby(['id_mutation'], sort=False)\
                    .agg({'valeur_fonciere':'mean', 'surface_reelle_bati': 'sum'})\
                    .rename(columns={'surface_reelle_bati':'surface_reelle_cumulee'})
surface_by_trans.head(10)

,valeur_fonciere,surface_reelle_cumulee
id_mutation,,
2014-63,1.0,460.0
2014-89,120000.0,1155.0
2014-96,175000.0,124.0
2014-99,70000.0,646.0
2014-101,90000.0,250.0
2014-123,750.0,12.0
2014-130,610000.0,72.0
2014-140,1.0,1307.0
2014-152,139800.0,323.0


In [26]:
#puis j'applique le filtre de 300m² qui s'applique desormais sur la nouvelle colonne sommé
surface_by_trans = surface_by_trans[surface_by_trans['surface_reelle_cumulee'] > 300]
surface_by_trans.head(10)

,valeur_fonciere,surface_reelle_cumulee
id_mutation,,
2014-63,1.0,460.0
2014-89,120000.0,1155.0
2014-99,70000.0,646.0
2014-140,1.0,1307.0
2014-152,139800.0,323.0
2014-267,284132.0,3136.0
2014-321,210000.0,635.0
2014-473,50700.0,832.0
2014-527,70000.0,602.0


In [27]:
# Jointure des 2 datafram
# cette jointure permettra indirectemnt d'appliqer les filtres de l'adrese sur le dataframe des données calculées
q2_filtered = address_by_trans.join(surface_by_trans, on='id_mutation', how='inner')
q2_filtered.head(10)

,id_mutation,adresse_numero,adresse_nom_voie,nom_commune,code_postal,code_commune,valeur_fonciere,surface_reelle_cumulee
2691163,2015-1145894,63,AV DES CHAMPS ELYSEES,Paris 8e Arrondissement,75008,75108,6000000.0,677.0
2691897,2015-1146366,102,AV DES CHAMPS ELYSEES,Paris 8e Arrondissement,75008,75108,41303366.0,842.0
2880379,2016-1234297,66,AV DES CHAMPS ELYSEES,Paris 8e Arrondissement,75008,75108,8983000.0,1175.0
2882219,2016-1235506,66,AV DES CHAMPS ELYSEES,Paris 8e Arrondissement,75008,75108,19404926.0,1763.0
2998574,2018-1276869,114,AV DES CHAMPS ELYSEES,Paris 8e Arrondissement,75008,75108,598015740.0,8455.0
2998892,2018-1277084,66,AV DES CHAMPS ELYSEES,Paris 8e Arrondissement,75008,75108,77000000.0,4584.0
3000713,2018-1278313,144,AV DES CHAMPS ELYSEES,Paris 8e Arrondissement,75008,75108,310600000.0,13834.0
3000778,2018-1278313,150,AV DES CHAMPS ELYSEES,Paris 8e Arrondissement,75008,75108,310600000.0,13834.0
3001776,2018-1278906,62,AV DES CHAMPS ELYSEES,Paris 8e Arrondissement,75008,75108,4500000.0,2211.0
997804,2019-433060,66,AV DES CHAMPS ELYSEES,Paris 8e Arrondissement,75008,75108,21000000.0,1941.0


In [0]:
# Select and aggregate the relevant data
# On va faire une moyenne des prix au mètre carré de toutes les transactions
sqm = q2_filtered['valeur_fonciere']/q2_filtered['surface_reelle_cumulee']

# Compute the average of the cost per square meter for each transaction
mean_cost_per_sq_meter = sqm.mean()

In [29]:
print(f'The average observed cost per square meter, for business bigger than 300 m² is {mean_cost_per_sq_meter:,.0f} €')

The average observed cost per square meter, for business bigger than 300 m² is 22,185 €


### Remarque :
Meme si ce n'est sans doute pas le cas sur Paris.
Pour une étude plus complète il faudrait faire attention aux transactions qui ont lieu pou r 1€ symbolique, car cela ppeut fortement faussé ces estimations.
    
Dans une logique de Data-Minig, il audrait sans doute exclure ces données.
    
**Exemple :**
    
| id_mutation | adresse_numero | adresse_nom_voie | nom_commune | code_postal | code_commune | surface_reelle_bati | valeur_fonciere |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 2018-24 | 5266 | SAINT AMOUZE | Montagnat | 1250 | 1254 | 917.0 | 1.0 |
| 2018-24 | 5266 | SAINT AMOUZE | Montagnat | 1250 | 1254 | 650.0 | 1.0 |


### Question 2b (3 points)

Your customer also wants to know how much money was needed for the most expensive transaction and the address of the property. Can you provide them that information?

In [30]:
# Your code here

# Pour avoir une adresse complète et pour mettre en oeuvre l'ajout de colonnes vus en cours
# On va créer une nouvelle colonne qui reprend l'adresse complète
q2_filtered['adresse_complete'] = q2_filtered['adresse_numero'] + ' ' + \
                                q2_filtered['adresse_nom_voie'] + ' - '+ \
                                q2_filtered['code_postal'] +' '+ q2_filtered['nom_commune']

q2_filtered.head(5)

,id_mutation,adresse_numero,adresse_nom_voie,nom_commune,code_postal,code_commune,valeur_fonciere,surface_reelle_cumulee,adresse_complete
2691163,2015-1145894,63,AV DES CHAMPS ELYSEES,Paris 8e Arrondissement,75008,75108,6000000.0,677.0,63 AV DES CHAMPS ELYSEES - 75008 Paris 8e Arro...
2691897,2015-1146366,102,AV DES CHAMPS ELYSEES,Paris 8e Arrondissement,75008,75108,41303366.0,842.0,102 AV DES CHAMPS ELYSEES - 75008 Paris 8e Arr...
2880379,2016-1234297,66,AV DES CHAMPS ELYSEES,Paris 8e Arrondissement,75008,75108,8983000.0,1175.0,66 AV DES CHAMPS ELYSEES - 75008 Paris 8e Arro...
2882219,2016-1235506,66,AV DES CHAMPS ELYSEES,Paris 8e Arrondissement,75008,75108,19404926.0,1763.0,66 AV DES CHAMPS ELYSEES - 75008 Paris 8e Arro...
2998574,2018-1276869,114,AV DES CHAMPS ELYSEES,Paris 8e Arrondissement,75008,75108,598015740.0,8455.0,114 AV DES CHAMPS ELYSEES - 75008 Paris 8e Arr...


In [31]:

max_trans = q2_filtered[q2_filtered['valeur_fonciere'] == q2_filtered['valeur_fonciere'].max()]
print(max_trans)

max_cost = int(max_trans['valeur_fonciere'].values)
address  = str(max_trans['adresse_complete'].values)

          id_mutation adresse_numero       adresse_nom_voie  \
2998574  2018-1276869            114  AV DES CHAMPS ELYSEES   

                     nom_commune code_postal code_commune  valeur_fonciere  \
2998574  Paris 8e Arrondissement       75008        75108      598015740.0   

         surface_reelle_cumulee  \
2998574                  8455.0   

                                          adresse_complete  
2998574  114 AV DES CHAMPS ELYSEES - 75008 Paris 8e Arr...  


In [32]:
print(f'The cost of the biggest sale transaction was {max_cost/1_000_000:,.0f} m€ for a property located at {address}')

The cost of the biggest sale transaction was 598 m€ for a property located at ['114 AV DES CHAMPS ELYSEES - 75008 Paris 8e Arrondissement']


### Question 2c (bonus: 1 point)
Can you tell what store is now located at the address found in your answer for question 2b?

The Store located *'114 AVENUE des CHAMPS ELYSEES, PARIS'* is **THE APPLE STORE**


<a href ="https://www.apple.com/fr/retail/champs-elysees/?cid=aos-fr-seo-maps"><img src="https://www.apple.com/fr/retail/champs-elysees/images/hero_thumb.jpg"></a>